### How to make api 

In [1]:
from wildcatpy.api_calls import * 

In [2]:
!python3 -m pip install python-dotenv


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

username = os.getenv("USERNAME") # you can also type your password here manually
password = os.getenv('PASSWORD') # You can also type your username here manually


In [4]:
api_call = WildcatApi(username,password)

In [5]:
# with this private func we can make a call so we can test new api's 

add = "/map/all/describe" # this is what has to be added to standard ->https://focus.sensingclues.org/api/
output = api_call._api_call("get", add, {}).json()
print(output)

{'$version': '1.3.3-beta', 'models': {'9999': {'id': '9999', 'name': '9999', 'description': 'Creekish mountains Upload', 'layers': [{'id': 0, 'name': 'test_polygon', 'description': 'All Polygon geometries for layer test_polygon', 'geometryType': 'Polygon'}, {'id': 1, 'name': 'test_multipolygon', 'description': 'All MultiPolygon geometries for layer test_multipolygon', 'geometryType': 'MultiPolygon'}]}, 'track': {'id': 'track', 'name': 'track', 'description': 'Track layers', 'layers': [{'id': 0, 'name': 'Tracks', 'description': 'All MultiPoint geometries for Track', 'geometryType': 'MultiPoint'}]}, 'default': {'id': 'default', 'name': 'default', 'description': 'Default Entity layers', 'layers': [{'id': 0, 'name': 'Observations', 'description': 'All Point geometries for Observations', 'geometryType': 'Point'}, {'id': 1, 'name': 'Tracks', 'description': 'All Point geometries for Track', 'geometryType': 'Point'}, {'id': 2, 'name': 'Agents', 'description': 'All Point geometries for Agent', 

In [6]:
# this is the json file I made to extract the correct values 
# What it does it it goes deeper in the dict (models) and starts looping if possible
# Per row it will extract the values in the extract_values argument
# It will explode every row with values that are in the explode_values
# What happens now it makes a new row for the default and for the tracks
# But both are containing the extract_values 
# We fix this afterwards by filtering. Normally we don't want seperated rows when exploding muliple times
{
  "cols_to_data": [],
  "extractor": {
    "extract_values": [
      "pid"
    ],
    "layers": {
      "explode_values": [
        "id",
        "name",
        "geometryType"
      ]
    }
  }
}

{'cols_to_data': [],
 'extractor': {'extract_values': ['pid'],
  'layers': {'explode_values': ['id', 'name', 'geometryType']}}}

In [7]:
# small change because the extractor needs a list from which it can start extracting
# The input data is a dict from which the key(pid) gives a dict with the info
# So we convert the dict to a list of dicts and add the pid as item in the list
# Then we can start iterating through this with the extractor beause it is the same strcuture as other calls
new_output = [{**{"pid": key}, **output["models"][key]} for key in output["models"].keys()]


In [8]:
extr = dataExtractor("all_layers")
extr_output = extr.extr(new_output)

In [9]:
col_trans = {
    "id": "lid"
}
df = pd.DataFrame(extr_output)\
       .rename(columns=col_trans)\
       .query("pid != 'track' and pid != 'default'")
#df = df.loc[~df["pid"].isin(["track", "default"])]


In [10]:
df.head()

,pid,lid,name,geometryType
0,9999,0,test_polygon,Polygon
1,9999,1,test_multipolygon,MultiPolygon


In [11]:
# this is a very simple one so we just make a standard private function in the class
# This should be a private function because it is only used by other functions and not by the users
# Private means that you start the function with a _  (also not visible in documenation by user then)

# To test this new function we extend the WildcatAPi Class 
# Then we make the variable url_addition
# Then use the _api_call function and provide the type request, url_addition and payload if needed
# This function returns all the models
# Normally we use an extraction but since this function returns almost everything it is not needed.
# If the function works here you can implement it in the api_calls.py

class testNewAPi(WildcatApi):
    def _get_all_layers(self):
        col_trans = {
            "id": "lid",
            "name": "layerName"
                }
        url_addition = "/map/all/describe"
        r = self._api_call("get", url_addition)
        output = r.json()
        new_output = [{**{"pid": key}, **output["models"][key]} for key in output["models"].keys()]
        extr = dataExtractor("all_layers")
        extr_output = extr.extr(new_output)
        return pd.DataFrame(extr_output)\
                 .rename(columns=col_trans)\
                 .query("pid != 'track' and pid != 'default'")
    



In [12]:
# run test if it works
# if so we can implement it 
test = testNewAPi(username,password)
test._get_all_layers()

,pid,lid,layerName,geometryType
0,9999,0,test_polygon,Polygon
1,9999,1,test_multipolygon,MultiPolygon


In [13]:
{
  "cols_to_data": ["features"],
  "extractor": {
    "extract_values": [
      "geometry",
      "type",
      "id",
      "properties"
    ]

  }
}

{'cols_to_data': ['features'],
 'extractor': {'extract_values': ['geometry', 'type', 'id', 'properties']}}

## Part 2 

In [14]:
#call again
# with this private func we can make a call so we can test new api's 
project_id = 9999
layer_id = 0
add = f"/map/all/{project_id}/{layer_id}/features/" # this is what has to be added to standard ->https://focus.sensingclues.org/api/
output = api_call._api_call("post", add, {}).json()

In [15]:
extr = dataExtractor("layer_details")
extr_output = extr.extr(output, nested_col_names=True)

In [16]:
output["features"][0]

{'type': 'Feature',
 'id': '186060457',
 'properties': {'DocId': '/GeoFeature/G9999-7265885820642820983-1.json',
  'ShortId': '186060457',
  'OBJECTID': '186060457',
  'EntityId': 'G9999-7265885820642820983-1',
  'NAME': None},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[5.87116241455078, 51.8538062484514],
    [5.88180541992188, 51.8646200195489],
    [5.88180541992188, 51.8771268259966],
    [5.84163665771484, 51.8894182626641],
    [5.81039428710938, 51.8817894907091],
    [5.8392333984375, 51.8622878573851],
    [5.87116241455078, 51.8538062484514]]]}}

In [17]:
pd.DataFrame(extr_output)

,_type,_Id,geometry_type,geometry_coordinates,properties_type,properties_coordinates
0,Feature,False,Polygon,"[[[5.87116241455078, 51.8538062484514], [5.881...",False,False
1,Feature,False,Polygon,"[[[5.85983276367188, 51.8494588041788], [5.836...",False,False
2,Feature,False,Polygon,"[[[4.87037658691406, 52.2530274427686], [4.801...",False,False
3,Feature,False,Polygon,"[[[5.06778717041016, 52.0571349329375], [5.067...",False,False
4,Feature,False,Polygon,"[[[5.48269271850586, 51.8294127159533], [5.463...",False,False
